
# Setting up NetGraf on Chameleon set-up for One Dashboard Setup for End-to-End Performance Monitoring

We develop monitoring dashbaord, using Grafana, using our Chameleon cloud set-up. We will start by (1) setting up nodes on Chicago site and Texas sites, then (2) install open-source network monitoring tools to allow a one dashboard perfomance monitoring for all nodes.



###  Setup NetPerf's Enviroment 

In [12]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='ProjName'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-uc-key
export SSH_PRIVATE_KEY=${HOME}/work/my-private-key

export NETWORK_SUBNET="192.168.100.0/24"
export UC_ALLOCATION_START="192.168.100.10"
export UC_ALLOCATION_END="192.168.100.19"
export TACC_ALLOCATION_START="192.168.100.20"
export TACC_ALLOCATION_END="192.168.100.29"

export RESOURCE_PREFIX="$USER-netperf-$(date +%b%d)"



### Set NetPerf Variables 
  - `OS_PROJECT_NAME`: The name of the project you want to use.
  - `UC_RESERVATION`: Reservation ID at UC.
  - `TACC_RESERVATION`: Reservation ID at TACC.
  - `SSH_KEY_NAME`: SSH key name as registered in Chameleon
  - `SSH_PRIVATE_KEY`: Path to the corresponding private key in your Jupyter container.
  - `NETWORK_SUBNET`: Private IP subnet that will span both Chameleon sites.
  - `UC_ALLOCATION_START`: The start of the IP range assigned at the UC site.
  - `UC_ALLOCATION_END`: The end of the IP range assigned at the UC site.
  - `TACC_ALLOCATION_START`: The start of the IP range assigned at the TACC site.
  - `TACC_ALLOCATION_END`: The start of the IP range assigned at the TACC site.
  - `RESOURCE_PREFIX`: A begin string used to idenify resources provision. This includes your user ID so that your resources can be distinguised from those belonging to other tutorial particpants.
  - `GENI_PEM_FILE`: The path to your GENI certificate file in you Jupyter container.
  - `UC_DIRECTSTITCH_VLAN`: The VLAN using to attach to the DirectStitch port at UC.
  - `TACC_DIRECTSTITCH_VLAN`: The VLAN using to attach to the DirectStitch port at TACC.

In [13]:
UC_RESERVATION_NAME=${RESOURCE_PREFIX}"-UC"

# Set the names for the network, subnet, router, and switch. 
UC_NETWORK_NAME=${RESOURCE_PREFIX}"-Network-UC"
UC_SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet-UC"
UC_ROUTER_NAME=${RESOURCE_PREFIX}"-Router-UC"

### Creating a setup VMs and network on the UC Chameleon Site

In [ ]:
OS_REGION_NAME='CHI@UC'

START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+5 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${EXOGENI_STITCH_NAME}
blazar lease-create \
   --physical-reservation min=1,max=3,resource_properties='["=", "$node_type", "compute_haswell"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${UC_NETWORK_NAME}",resource_properties='["==","$physical_network","exogeni"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${UC_RESERVATION_NAME}

UC_RESERVATION=`blazar lease-show --format value -c id ${UC_RESERVATION_NAME}`
echo UC_RESERVATION $UC_RESERVATION

UC_NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${UC_RESERVATION_NAME} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo UC_NODE_RESERVATION $UC_NODE_RESERVATION

In [15]:
OS_REGION_NAME='CHI@UC'

UC_DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${UC_NETWORK_NAME}`
echo UC DirectStitch VLAN: $UC_DIRECTSTITCH_VLAN 

UC_FLOATING_IP=`lease_list_floating_ips $UC_RESERVATION`
echo UC_FLOATING_IP $UC_FLOATING_IP

UC DirectStitch VLAN: 3291
UC_FLOATING_IP 192.5.87.184


In [ ]:
OS_REGION_NAME='CHI@UC'
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --allocation-pool start=${UC_ALLOCATION_START},end=${UC_ALLOCATION_END} \
                        --dhcp \
                        --network ${UC_NETWORK_NAME} \
                        ${UC_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${UC_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${UC_ROUTER_NAME} ${UC_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${UC_ROUTER_NAME}

echo Network ${UC_NETWORK_NAME} is ready for nodes!

In [ ]:
UC_NODE_NAME=${RESOURCE_PREFIX}-node

OS_REGION_NAME='CHI@UC'
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-Ubuntu16.04" \
                        --key-name "uc-key" \
                        --hint reservation=${UC_NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${UC_NETWORK_NAME} \
                        ${UC_NODE_NAME}

In [18]:
OS_REGION_NAME='CHI@UC'
#Attach the floating IP address to the server
echo $UC_FLOATING_IP 
openstack server add floating ip $UC_NODE_NAME $UC_FLOATING_IP

192.5.87.184


### Creating a VM and network on the TACC Chameleon Site
 We simply repeat the above steps at TACC

In [19]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='Projname'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-tacc-key
export SSH_PRIVATE_KEY=${HOME}/work/my-private-key

export NETWORK_SUBNET="192.168.100.0/24"
export UC_ALLOCATION_START="192.168.100.10"
export UC_ALLOCATION_END="192.168.100.19"
export TACC_ALLOCATION_START="192.168.100.20"
export TACC_ALLOCATION_END="192.168.100.29"

export RESOURCE_PREFIX="$USER-netperf-$(date +%b%d)"



In [20]:
OS_REGION_NAME='CHI@TACC'

TACC_RESERVATION_NAME=${RESOURCE_PREFIX}"-TACC"

# Set the names for the network, subnet, router, and switch. 
# See above about using identifiable names.  
TACC_NETWORK_NAME=${RESOURCE_PREFIX}"-Network-TACC"
TACC_SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet-TACC"
TACC_ROUTER_NAME=${RESOURCE_PREFIX}"-Router-TACC"

In [ ]:
OS_REGION_NAME='CHI@TACC'

START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+5 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${EXOGENI_STITCH_NAME}
blazar lease-create \
   --physical-reservation min=1,max=3,resource_properties='["=", "$node_type", "compute_haswell"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${TACC_NETWORK_NAME}",resource_properties='["==","$physical_network","physnet"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${TACC_RESERVATION_NAME}

TACC_RESERVATION=`blazar lease-show --format value -c id ${TACC_RESERVATION_NAME}`
echo TACC_RESERVATION $TACC_RESERVATION

TACC_NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${TACC_RESERVATION} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo TACC_NODE_RESERVATION $TACC_NODE_RESERVATION
   
   

In [ ]:
OS_REGION_NAME='CHI@TACC'
TACC_DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${TACC_NETWORK_NAME}`
echo TACC DirectStitch VLAN: $TACC_DIRECTSTITCH_VLAN 

TACC_FLOATING_IP=`lease_list_floating_ips $TACC_RESERVATION`
echo TACC_FLOATING_IP $TACC_FLOATING_IP 


In [ ]:
OS_REGION_NAME='CHI@TACC'
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --allocation-pool start=${TACC_ALLOCATION_START},end=${TACC_ALLOCATION_END} \
                        --dhcp \
                        --network ${TACC_NETWORK_NAME} \
                        ${TACC_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${TACC_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${TACC_ROUTER_NAME} ${TACC_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${TACC_ROUTER_NAME}

echo Network ${TACC_NETWORK_NAME} is ready for nodes!

In [ ]:
echo TACC_RESERVATION $TACC_RESERVATION

TACC_NODE_NAME=${RESOURCE_PREFIX}-node

OS_REGION_NAME='CHI@TACC'
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-Ubuntu16.04"\
                        --key-name "tacc-key" \
                        --hint reservation=${TACC_NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${TACC_NETWORK_NAME} \
                        ${TACC_NODE_NAME}


In [ ]:
OS_REGION_NAME='CHI@TACC'

#Attach the floating IP address to the server
echo $TACC_FLOATING_IP 
openstack server add floating ip $TACC_NODE_NAME $TACC_FLOATING_IP

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='Projname'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-tacc-key
export SSH_PRIVATE_KEY=${HOME}/work/my-private-key

export NETWORK_SUBNET="192.168.100.0/24"
export UC_ALLOCATION_START="192.168.100.10"
export UC_ALLOCATION_END="192.168.100.19"
export TACC_ALLOCATION_START="192.168.100.20"
export TACC_ALLOCATION_END="192.168.100.29"

export RESOURCE_PREFIX="$USER-myproject-$(date +%b%d)"



In [ ]:
xoStitch create -sp1 uc -vlan1 3293 -sp2 tacc -vlan2 3504 -c geni-pemfile.pem

In [ ]:
pwd

### Using ExoGENI to connect ports using direct stitching

In [ ]:
echo UC DirectStitch VLAN:   $UC_DIRECTSTITCH_VLAN
echo TACC DirectStitch VLAN: $TACC_DIRECTSTITCH_VLAN
xoStitch create -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE